# City street network orientations

Compare the spatial orientations of city street networks.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

In [31]:
# define the study sites as label : query
places = {'Wellington':'Wellington City, New Zealand',
          'Christchurch':'Christchurch, New Zealand',
          'Hamilton':'Hamilton City, New Zealand',
          'Tauranga':'Tauranga City, New Zealand',
          'Napier':'Napier City, New Zealand',
          'Hastings':'Hastings District, New Zealand',
          'Dunedin':'Dunedin City, New Zealand',
          'Palmerston North':'Palmerston North City, New Zealand',
          'Nelson':'Nelson , New Zealand',
          'Rotorua':'Rotorua District, New Zealand',
          'Whangarei':'Whangarei District, New Zealand',
          'New Plymouth':'New Plymouth District, New Zealand',
          'Invercargill':'Invercargill City, New Zealand',
          'Whanganui':'Wanganui District, New Zealand'
          }

# for cities with second results
places2 = {'Auckland'       : 'Auckland, New Zealand',
           'Gisborne'       : 'Gisborne, New Zealand'
          }

In [32]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values())

for place in places2.values():
    gdfAdd=ox.gdf_from_place(place, which_result=2)
    gdf = pd.concat([gdf, gdfAdd])
    
gdf

,bbox_east,bbox_north,bbox_south,bbox_west,geometry,place_name
0,174.895410,-41.143540,-41.362380,174.613106,"POLYGON ((174.6131062 -41.2842883, 174.6131598...","Wellington City, Wellington, New Zealand/Aotearoa"
1,172.821627,-43.389087,-43.629201,172.393025,"POLYGON ((172.3930248 -43.4633402, 172.3938705...","Christchurch, Christchurch City, Canterbury, N..."
2,175.344877,-37.699330,-37.846039,175.184301,"POLYGON ((175.1843007 -37.7429921, 175.1852557...","Hamilton City, Waikato, New Zealand/Aotearoa"
3,176.415043,-37.623180,-37.770565,176.074108,"POLYGON ((176.0741079 -37.7566484, 176.0743984...","Tauranga City, Bay of Plenty, New Zealand/Aote..."
4,176.925930,-39.388193,-39.570465,176.811266,"POLYGON ((176.8112659 -39.5516378, 176.8115245...","Napier City, Hawke's Bay, New Zealand/Aotearoa"
5,177.096017,-38.855956,-39.892627,176.142726,"POLYGON ((176.1427264 -39.5846828, 176.1471686...","Hastings District, Hawke's Bay, New Zealand/Ao..."
6,170.750885,-45.228967,-46.051927,169.736478,"POLYGON ((169.7364779 -45.7137127, 169.7365619...","Dunedin City, Otago, New Zealand/Aotearoa"
7,175.790592,-40.268077,-40.529685,175.497335,"POLYGON ((175.4973346 -40.4338647, 175.498545 ...","Palmerston North City, Manawatu-Wanganui, New ..."
8,173.598762,-40.804890,-41.393427,173.176422,"POLYGON ((173.1764224 -41.304308, 173.1988874 ...","Nelson, New Zealand/Aotearoa"
9,176.615174,-37.936963,-38.568698,175.981747,"POLYGON ((175.9817466 -38.1385781, 175.9843278...","Rotorua District, Bay of Plenty, New Zealand/A..."


## Get the street networks and their edge bearings

In [33]:
bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    G = ox.add_edge_bearings(G)
    
    if weight_by_length:
        # weight bearings by length (meters)
        streets = [(d['bearing'], int(d['length'])) for u, v, k, d in G.edges(keys=True, data=True)]
        city_bearings = []
        for street in streets:
            city_bearings.extend([street[0]] * street[1])
        bearings[place] = pd.Series(city_bearings)
    else:
        # don't weight bearings, just take one value per street segment
        bearings[place] = pd.Series([data['bearing'] for u, v, k, data in G.edges(keys=True, data=True)])
        

for place in sorted(places2.keys()):
    
    # get the graph
    query = places2[place]
    G = ox.graph_from_place(query, network_type='drive', which_result=2)
    
    # calculate edge bearings
    G = ox.add_edge_bearings(G)
    
    if weight_by_length:
        # weight bearings by length (meters)
        streets = [(d['bearing'], int(d['length'])) for u, v, k, d in G.edges(keys=True, data=True)]
        city_bearings = []
        for street in streets:
            city_bearings.extend([street[0]] * street[1])
        bearings[place] = pd.Series(city_bearings)
    else:
        # don't weight bearings, just take one value per street segment
        bearings[place] = pd.Series([data['bearing'] for u, v, k, data in G.edges(keys=True, data=True)])

## Visualize it

In [34]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = [ang * 360 / n for ang in range(0, n + 1)]
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Calibri', 'size':26, 'weight':'bold'}
    xtick_font = {'family':'Calibri', 'size':12, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Calibri', 'size':10, 'weight':'bold', 'alpha':0.3, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [35]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # this prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = [ang * 360 / n for ang in range(0, n + 1)]
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = count.tolist()
    count = [count[-1]] + count[:-1]

    count_merged = []
    count_iter = iter(count)
    for count in count_iter:
        merged_count = count + next(count_iter)
        count_merged.append(merged_count)

    return np.array(count_merged)

In [53]:
# create figure and axes
n = len(bearings)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})
axes = [item for sublist in axes for item in sublist]

combo_places = {**places,**places2}

# plot each city's polar histogram
for ax, place in zip(axes, sorted(combo_places.keys())):
    polar_plot(ax, bearings[place], title=place)

# add super title and save full image
suptitle_font = {'family':'Open Sans', 'fontsize':60, 'fontweight':'light', 'y':1.07}
fig.suptitle('Street Network Orientations: Aotearoa New Zealand', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
plt.gcf().savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()

In [45]:
# save network graphs
for name, query in places.items():
    fname="temp-"+name
    G=ox.graph_from_place(query, network_type='drive')
    ox.plot_graph(G, show=False, save=True, filename=fname, node_size=0)
    
for name, query in places2.items():
    fname="temp-"+name
    G=ox.graph_from_place(query, network_type='drive', which_result=2)
    ox.plot_graph(G, show=False, save=True, filename=fname, node_size=0)

In [52]:
import PIL

# create figure and axes
n = len(bearings)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
axes = [item for sublist in axes for item in sublist]

title_font = {'family':'Calibri', 'size':26, 'weight':'bold'}

# plot each city's network
for ax, place in zip(axes, sorted(combo_places.keys())):
    fname = "images/temp-"+place+".png"
    img = PIL.Image.open(fname)
    ax.axis('off')
    ax.set_title(place.upper(), y=1.05, fontdict=title_font)
    ax.imshow(img)
    
# add super title and save full image
suptitle_font = {'family':'Open Sans', 'fontsize':60, 'fontweight':'light', 'y':1.07}
fig.suptitle('City Street Networks: Aotearoa New Zealand', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
plt.gcf().savefig('images/street-orientations-streets.png', dpi=120, bbox_inches='tight')
plt.close()